<a href="https://colab.research.google.com/github/BardRimon/Study/blob/main/Introduction_in_NLP/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%9A%D0%BB%D1%8E%D0%BA%D0%B8%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание:
1. Объяснить значение трех UD-тегов синтаксических отношений на примере фрагмента размеченного корпуса.
2. Написать функцию разбиения сложносочиненного предложения из двух частей на простые.
3. Написать функцию нахождения наименьшего общего предка двух токенов в дереве зависимостей.
4. Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами.


## задание 1 Ud-теги

 они же universal dependency тэги

Виды:

| UD-тег  | Полное название               | Значение / описание                                      |
|---------|------------------------------|---------------------------------------------------------|
| nsubj   | nominal subject              | Подлежащее - слово, выполняющее действие (связь с глаголом) |
| obj     | direct object               | Прямое дополнение - объект действия глагола             |
| amod    | adjectival modifier         | Определение - прилагательное или причастие, описывающее существительное |
| advmod  | adverbial modifier          | Обстоятельство - наречие, модифицирующее глагол, прилагательное или другое наречие |
| det     | determiner                  | Определитель - артикль, местоимение-определение         |
| case    | case marking                | Падежный элемент - предлог или падежный суффикс         |
| nmod    | nominal modifier            | Именной модификатор - зависимое существительное или группа, уточняющая другое существительное |
| root    | root                        | Корень - главное слово предложения (обычно сказуемое)   |
| aux     | auxiliary                   | Вспомогательный глагол                                   |
| conj    | conjunct                    | Союзное слово в составе сочинённого союза                |
| cc      | coordinating conjunction   | Координирующий союз                                      |
| punct   | punctuation                 | Знак препинания                                         |




In [2]:
!pip install -U spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 75.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [34]:

import spacy
from spacy import displacy
nlp = spacy.load("ru_core_news_sm")
text = "Было тихо, и над деревней стоял теплый туман."
print(text)
doc = nlp(text)



print("\n1. Тег: nsubj (Nominal subject) - Подлежащее")
print("Определение: Именная группа, которая является подлежащим глагола.")
print("Примеры из предложения:")
for token in doc:
    if token.dep_ == "nsubj":
        print(f"  Токен: '{token.text}' (лемма: '{token.lemma_}') -> Зависит от: '{token.head.text}'")


print("\n2. Тег: obj (Object) - Прямое дополнение")
print("Определение: Именная группа, которая является прямым дополнением глагола.")
print("Примеры из предложения:")
for token in doc:
    if token.dep_ == "obj":
        print(f"  Токен: '{token.text}' (лемма: '{token.lemma_}') -> Зависит от: '{token.head.text}'")

print("\n3. Тег: root (Root) - Корень предложения")
print("Определение: Глагол (или другое слово), которое является корнем предложения (главное слово, от которого зависят остальные).")
print("Примеры из предложения:")
for token in doc:
    if token.dep_ == "root":
        print(f"  Токен: '{token.text}' (лемма: '{token.lemma_}')")


print("\nПример зависимостей для всех токенов в предложении:")
for token in doc:
    print(f"Токен: '{token.text}', Зависимость: '{token.dep_}', Голова: '{token.head.text}'")


Было тихо, и над деревней стоял теплый туман.

1. Тег: nsubj (Nominal subject) - Подлежащее
Определение: Именная группа, которая является подлежащим глагола.
Примеры из предложения:
  Токен: 'туман' (лемма: 'туман') -> Зависит от: 'стоял'

2. Тег: obj (Object) - Прямое дополнение
Определение: Именная группа, которая является прямым дополнением глагола.
Примеры из предложения:

3. Тег: root (Root) - Корень предложения
Определение: Глагол (или другое слово), которое является корнем предложения (главное слово, от которого зависят остальные).
Примеры из предложения:

Пример зависимостей для всех токенов в предложении:
Токен: 'Было', Зависимость: 'cop', Голова: 'тихо'
Токен: 'тихо', Зависимость: 'ROOT', Голова: 'тихо'
Токен: ',', Зависимость: 'punct', Голова: 'стоял'
Токен: 'и', Зависимость: 'cc', Голова: 'стоял'
Токен: 'над', Зависимость: 'case', Голова: 'деревней'
Токен: 'деревней', Зависимость: 'obl', Голова: 'стоял'
Токен: 'стоял', Зависимость: 'conj', Голова: 'тихо'
Токен: 'теплый', За

## Задание 2: разбиваем сложносочинённое предложение на простые
Написать функцию разбиения сложносочиненного предложения из двух частей на простые.

В русском языке зачастую это сочетание сс - союза и punct пунктуации

In [12]:
def split_compound_sentence(sentence):

    doc = nlp(sentence)
    simple_sentences = []
    split_index = -1


    for i, token in enumerate(doc):
        if token.dep_ == "cc" or token.dep_ == 'mark':
            split_index = i
            break

    if split_index != -1:

        sent1_tokens = doc[:split_index - 1]
        sentence1 = "".join(token.text_with_ws for token in sent1_tokens).strip()

        sent2_tokens = doc[split_index + 1:]
        sentence2 = "".join(token.text_with_ws for token in sent2_tokens).strip()
        # починить заглавную букву
        sentence1 = sentence1[0].upper() + sentence1[1:]
        sentence2 = sentence2[0].upper() + sentence2[1:]


        if not sentence1.endswith(('.', '!', '?')):
             sentence1 += '.'
        if not sentence2.endswith(('.', '!', '?')):
             sentence2 += '.'


        simple_sentences.append(sentence1)
        simple_sentences.append(sentence2)
    else:
        # Если союз не найден, возвращаем исходное предложение как одно целое
        simple_sentences.append(sentence)

    return simple_sentences


sentence1 = "Было тихо, и над деревней стоял теплый туман."
sentence2 = "Я пошел в магазин, но он был закрыт."
sentence3 = "Я в этот мир пришёл, чтобы видеть солнце"
sentence4 = "Простое предложение без союза."

print(f"\nИсходное предложение: '{sentence1}'")
split_sentences1 = split_compound_sentence(sentence1)
print(f"Разбитые предложения: {split_sentences1}")

print(f"\nИсходное предложение: '{sentence2}'")
split_sentences2 = split_compound_sentence(sentence2)
print(f"Разбитые предложения: {split_sentences2}")

print(f"\nИсходное предложение: '{sentence3}'")
split_sentences3 = split_compound_sentence(sentence3)
print(f"Разбитые предложения: {split_sentences3}")

print(f"\nИсходное предложение: '{sentence4}'")
split_sentences4 = split_compound_sentence(sentence4)
print(f"Разбитые предложения: {split_sentences4}")




Исходное предложение: 'Было тихо, и над деревней стоял теплый туман.'
Разбитые предложения: ['Было тихо.', 'Над деревней стоял теплый туман.']

Исходное предложение: 'Я пошел в магазин, но он был закрыт.'
Разбитые предложения: ['Я пошел в магазин.', 'Он был закрыт.']

Исходное предложение: 'Я в этот мир пришёл, чтобы видеть солнце'
Разбитые предложения: ['Я в этот мир пришёл.', 'Видеть солнце.']

Исходное предложение: 'Простое предложение без союза.'
Разбитые предложения: ['Простое предложение без союза.']


## Задание 3: функция нахождения наименьшего предка
Написать функцию нахождения наименьшего общего предка двух токенов в дереве зависимостей.



In [39]:
text = "Прислонясь к дверному косяку я ловлю в далёком отголоске, что случиться на моём веку"
doc = nlp(text)
# print(token1.dep_, token1.head)
displacy.render(doc, style="dep", options={"distance": 100})


def get_ancestors(token):

    path = []
    current = token
    while current:
        path.append(current)
        if current.head == current:
            break
        current = current.head
    return path

def lca(token1, token2):

    ancestors1 = get_ancestors(token1)
    ancestors2 = get_ancestors(token2)
    for t1 in reversed(ancestors1):
        if t1 in ancestors2:
            return t1
    return None

# Пример использования
token1 = doc[4]
token2 = doc[8]

common_ancestor = lca(token1, token2)
print(f"Наименьший общий предок: {common_ancestor.text} ({common_ancestor.pos_})")



Наименьший общий предок: ловлю (VERB)


## Задание 4:сравнить сравнение zss и cos
Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами.